<a href="https://colab.research.google.com/github/Kyle950204/KOREA_ZOMBIE-FILM-ANALYSIS/blob/main/%EC%98%81%ED%99%94_%EC%9B%94%EB%93%9C%EC%9B%8CZ_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [1]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
     |████████████████████████████████| 460kB 36.6MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# konlpy를 실행하기
import konlpy

In [3]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [4]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
## 문서 ID로 실제 파일 불러오기
# 부산행 58000건.csv
## https://drive.google.com/file/d/1aVWboIGX9wvso7VOlnriEBpDUxdu05hF/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1aVWboIGX9wvso7VOlnriEBpDUxdu05hF'})
rawdata_downloaded.GetContentFile('월드워Z 12688건.csv')

In [6]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('월드워Z 12688건.csv', sep=",")

In [7]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,runs****,조조보고옴. 브래드 피트는 나이가 들어도 멋지다. 스케일과 비교적 깔끔한 전개만으로...,10,1863,237,2013-06-20 12:46:00,0
1,1,여긴어디(pdk4****),진심 두번 빡친다 마누라가 전화걸때 이슬람그년 마이크 잡을때 아오!,10,1580,94,2013-06-28 01:07:00,0
2,2,웃는중(nsju****),은위 빠돌이들 별테러 방지용 댓글,10,1139,128,2013-06-20 10:07:00,0
3,3,해롱2(qkrq****),아오 노래부르지말라고 노래부르지마 OO들아,10,1011,25,2013-06-24 23:38:00,0
4,4,whan97(cmcm****),김수현이 브래드피트보다 연기 잘한다고 개소리찌껄이는셋기는뭐냐 OO인가,10,939,65,2013-06-20 17:04:00,0
...,...,...,...,...,...,...,...,...
12675,12675,쫑마리(no1m****),어떤게 재밌다는거지....,1,3,23,2013-07-09 12:01:00,0
12676,12676,허허(nomu****),광주사태를 간접적으로 체험했습니다 역시 좀비는 무서워요. 개인적으로 촛불든 좀비...,10,2,23,2013-06-30 10:46:00,0
12677,12677,JeongWoo(jw_h****),와.. 이건.. 대체.. 뭔지.. 여자친구랑 같이 봤는데. 발차기만 하더라고요!오우...,1,4,26,2013-07-08 15:29:00,0
12678,12678,푸른하늘(chan****),"책보고나서 영화를 봤더니 대실망이네요.. ㅡ,.ㅡ",1,3,27,2013-07-09 09:08:00,0


In [8]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [9]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,runs****,조조보고옴. 브래드 피트는 나이가 들어도 멋지다. 스케일과 비교적 깔끔한 전개만으로...,10,1863,237,2013-06-20 12:46:00,0
1,1,여긴어디(pdk4****),진심 두번 빡친다 마누라가 전화걸때 이슬람그년 마이크 잡을때 아오!,10,1580,94,2013-06-28 01:07:00,0
2,2,웃는중(nsju****),은위 빠돌이들 별테러 방지용 댓글,10,1139,128,2013-06-20 10:07:00,0
3,3,해롱2(qkrq****),아오 노래부르지말라고 노래부르지마 OO들아,10,1011,25,2013-06-24 23:38:00,0
4,4,whan97(cmcm****),김수현이 브래드피트보다 연기 잘한다고 개소리찌껄이는셋기는뭐냐 OO인가,10,939,65,2013-06-20 17:04:00,0
...,...,...,...,...,...,...,...,...
12675,12675,쫑마리(no1m****),어떤게 재밌다는거지....,1,3,23,2013-07-09 12:01:00,0
12676,12676,허허(nomu****),광주사태를 간접적으로 체험했습니다 역시 좀비는 무서워요. 개인적으로 촛불든 좀비...,10,2,23,2013-06-30 10:46:00,0
12677,12677,JeongWoo(jw_h****),와.. 이건.. 대체.. 뭔지.. 여자친구랑 같이 봤는데. 발차기만 하더라고요!오우...,1,4,26,2013-07-08 15:29:00,0
12678,12678,푸른하늘(chan****),"책보고나서 영화를 봤더니 대실망이네요.. ㅡ,.ㅡ",1,3,27,2013-07-09 09:08:00,0


# 형태소 분석

In [10]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [11]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [12]:
# "형태소" 변수의 저장 내용 확인
형태소

[('재미없었음', 'Adjective'), ('..', 'Punctuation'), ('ㅠ', 'KoreanParticle')]

In [13]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,재미없었음,Adjective,1,1
1,..,Punctuation,1,1
2,ㅠ,KoreanParticle,1,1


In [14]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[         0               1  type  count
 0       조조            Noun    10      1
 1      보고옴            Verb    10      1
 2        .     Punctuation    10      1
 3      브래드            Noun    10      1
 4       피트            Noun    10      1
 5        는            Josa    10      1
 6       나이            Noun    10      1
 7        가            Josa    10      1
 8      들어도            Verb    10      1
 9      멋지다       Adjective    10      1
 10       .     Punctuation    10      1
 11     스케일            Noun    10      1
 12       과            Josa    10      1
 13      비교            Noun    10      1
 14       적          Suffix    10      1
 15     깔끔한       Adjective    10      1
 16      전개            Noun    10      1
 17    만으로도            Josa    10      1
 18      좀비            Noun    10      1
 19      영화            Noun    10      1
 20       중          Suffix    10      1
 21      수작            Noun    10      1
 22      으로            Josa    10      1
 23     남을듯     

In [15]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [16]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,조조,Noun,10,1
1,보고옴,Verb,10,1
2,.,Punctuation,10,1
3,브래드,Noun,10,1
4,피트,Noun,10,1
...,...,...,...,...
12,",.",Punctuation,1,1
13,ㅡ,KoreanParticle,1,1
0,재미없었음,Adjective,1,1
1,..,Punctuation,1,1


In [17]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [18]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,조조,Noun,10,1
1,보고옴,Verb,10,1
2,.,Punctuation,10,1
3,브래드,Noun,10,1
4,피트,Noun,10,1
...,...,...,...,...
12,",.",Punctuation,1,1
13,ㅡ,KoreanParticle,1,1
0,재미없었음,Adjective,1,1
1,..,Punctuation,1,1


In [19]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [20]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소      품사           분류
!        Punctuation  1     21
                      2      4
                      3     11
                      4      6
                      5      8
                            ..
힘주고      Verb         10     2
힘차게나가다가  Adjective    1      1
힝        Noun         3      1
                      10     1
힣히하히흫    Noun         10     1
Name: 카운트, Length: 45213, dtype: int64

In [21]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [22]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소     품사          분류     
!       Punctuation 1    21
                    2     4
                    3    11
                    4     6
                    5     8
...                     ...
힘주고     Verb        10    2
힘차게나가다가 Adjective   1     1
힝       Noun        3     1
                    10    1
힣히하히흫   Noun        10    1

[45213 rows x 1 columns]

In [23]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')